In [ ]:
import numpy as np
import random
from collections import deque
import gymnasium as gym
from gymnasium import spaces
import tensorflow as tf
from tensorflow.keras import Model, layers
import pandas as pd
import os



2025-04-08 20:09:32.016683: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-08 20:09:32.023528: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-08 20:09:32.046770: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744157372.083048   19890 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744157372.090836   19890 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744157372.111831   19890 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [ ]:
action_array = []

In [ ]:
class StockTradingEnv(gym.Env):
    def __init__(self, df: pd.DataFrame, window_size: int=10, initial_balance=100000):
        self.df = df.reset_index()
        self.window_size = window_size
        self.balance = initial_balance

        obs_shape = (window_size + 2,)  # window_size prices + cash + holdings
        self.observation_space = spaces.Dict({
            "calls": spaces.Box(low=-np.inf, high=np.inf, shape=obs_shape, dtype=np.float32),
            "puts": spaces.Box(low=-np.inf, high=np.inf, shape=obs_shape, dtype=np.float32),
            "stock": spaces.Box(low=-np.inf, high=np.inf, shape=obs_shape, dtype=np.float32),
        })

        self.action_space = spaces.Dict({
            "calls": spaces.Discrete(51, start=-25),
            "puts": spaces.Discrete(51, start=-25),
        })
        
        ## reset(): ruthesh
        
        ## _get_obs(): ishu
        
        ## step(): apoorv, shriram

    # TODO: initialize and reset calls_held, puts_held, current_step
    def step(self, action):
        # action will be dict of calls, puts (integers from -25 to 25, inclusive)
        
        # get_features is not implemented yet
        (call_price, put_price, stock_price) = get_features()
        
        # adjust calls
        self.balance -= (action["calls"] * call_price)
        self.calls_held += action["calls"]
        
        # adjust puts
        self.balance -= (action["puts"] * put_price)
        self.puts_held += action["puts"]
        
        # calculate reward (get_reward not implemented yet)
        reward = get_reward()
        
        # next step
        self.curr_step += 1
        done = self.curr_step + self.window_size >= len(self.df)
        
        # get new observation
        new_obs = self._get_obs()
        
        return new_obs, reward, done
        
            ## within step: get_features(): Baz 
            ## returns: stock_price, curr_portfolio (tuple), call_price, put_price 
        ## render(): jerry
        ## compilation: jerry 



In [ ]:
class StockTradingEnv(gym.Env):
    def __init__(self, df, window_size=10, initial_balance=10000):
        super().__init__()
        self.df = df.reset_index(drop=True)
        self.window_size = window_size
        self.initial_balance = initial_balance

        # Action: scalar ∈ [-1, 1] (negative = sell, positive = buy)
        self.action_space = spaces.Box(low=-1.0, high=1.0, shape=(1,), dtype=np.float32)

        # State: unbounded (e.g., price history + position)
        obs_shape = (window_size + 2,)  # window_size prices + cash + holdings
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=obs_shape, dtype=np.float32)

        self.reset()

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.current_step = self.window_size
        self.balance = self.initial_balance
        self.shares_held = 0.0
        self.total_asset = self.initial_balance
        self.done = False
        return self._get_obs(), {}

    def _get_obs(self):
        window_prices = self.df['Close'].iloc[self.current_step - self.window_size:self.current_step].values
        return np.array(list(window_prices) + [self.balance, self.shares_held], dtype=np.float32)

    def step(self, action):
        if self.done:
            return self._get_obs(), 0.0, True, True, {}

        price = self.df['Close'].iloc[self.current_step]
        action_scalar = np.clip(action[0], -1, 1)

        # Scale action by balance
        if action_scalar > 0:
            # Buy
            amount_to_spend = self.balance * action_scalar
            shares_bought = amount_to_spend / price
            self.shares_held += shares_bought
            self.balance -= shares_bought * price
        else:
            # Sell
            shares_to_sell = self.shares_held * (-action_scalar)
            self.shares_held -= shares_to_sell
            self.balance += shares_to_sell * price

        self.total_asset = self.balance + self.shares_held * price
        reward = self.total_asset - self.initial_balance  # P&L since reset

        self.current_step += 1
        self.done = self.current_step >= len(self.df)

        return self._get_obs(), reward, self.done, False, {}

    def render(self):
        print(f"Step: {self.current_step}, Price: {self.df['Close'].iloc[self.current_step-1]:.2f}, "
              f"Balance: {self.balance:.2f}, Holdings: {self.shares_held:.2f}, "
              f"Total Asset: {self.total_asset:.2f}")

    def close(self):
        pass
